In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import os

In [2]:
os.makedirs("data_warehouse", exist_ok=True)

In [4]:
doctors_df=pd.read_csv("doctors_info.csv")
feedback_df=pd.read_json("patient_feedback.json")
patients_df=pd.read_csv("patients_data_with_doctor.csv")

In [5]:

def normalize_cols(df):
    df.columns = (
        df.columns.str.strip()
                  .str.lower()
                  .str.replace(" ", "_")
    )
    return df

patients_df = normalize_cols(patients_df)
doctors_df  = normalize_cols(doctors_df)
feedback_df = normalize_cols(feedback_df)

rename_map_patients = {
     "patientid": "patient_id",
     "doctorid": "doctor_id",
}
rename_map_doctors = {
     "id": "doctor_id",
     "name": "doctor_name",
}
rename_map_feedback = {
     "feedback": "patient_feedback_score",
}
patients_df = patients_df.rename(columns=rename_map_patients)
doctors_df  = doctors_df.rename(columns=rename_map_doctors)
feedback_df = feedback_df.rename(columns=rename_map_feedback)


required_patients = {"patient_id", "doctor_id"}
required_doctors  = {"doctor_id"}
required_feedback = {"patient_id"}

for need, df, name in [
    (required_patients, patients_df, "patients_df"),
    (required_doctors,  doctors_df,  "doctors_df"),
    (required_feedback, feedback_df, "feedback_df"),
]:
    missing = need - set(df.columns)
    if missing:
        raise ValueError(f"Missing columns {missing} in {name}. Please update the rename maps.")

if "patient_feedback_score" in feedback_df.columns:
    feedback_df["patient_feedback_score"] = pd.to_numeric(
        feedback_df["patient_feedback_score"], errors="coerce"
    )

patients_df = patients_df.drop_duplicates()
doctors_df  = doctors_df.drop_duplicates()
feedback_df = feedback_df.drop_duplicates()


In [6]:
patients_df.head()

,treatment_id,patient_id,doctor_id,treatment_type,treatment_cost,room_cost,treatment_date
0,T0001,P0067,D006,part,2723.53,1932.58,2025-08-04
1,T0002,P0006,D022,machine,3963.16,2215.53,2024-09-29
2,T0003,P0637,D014,seat,4108.71,4958.21,2024-09-14
3,T0004,P0314,D003,star,4793.77,1614.28,2024-09-25
4,T0005,P0326,D012,art,4138.91,2779.05,2024-10-11


In [7]:
doctors_df.head()

,doctor_id,doctor_name,specialty
0,D001,Cheyenne Wilson,Drilling engineer
1,D002,Taylor Potts,Pharmacologist
2,D003,Daniel Frye,Metallurgist
3,D004,Crystal West,Patent examiner
4,D005,Jennifer Smith,Tour manager


In [8]:
feedback_df.head()

,patient_id,treatment_id,patient_feedback_score,review_date
0,P0001,T052,3.8,2025-07-30
1,P0002,T093,4.3,2025-05-20
2,P0003,T015,1.4,2025-07-29
3,P0004,T072,2.0,2025-01-24
4,P0005,T061,1.6,2025-05-19


In [9]:

patients_doctors_df = patients_df.merge(
    doctors_df, on="doctor_id", how="left", validate="m:1"
)

final_df = patients_doctors_df.merge(
    feedback_df, on="patient_id", how="left", validate="m:1"
)

print("Merged shape:", final_df.shape)


Merged shape: (1000, 12)


In [10]:
final_df.head()

,treatment_id_x,patient_id,doctor_id,treatment_type,treatment_cost,room_cost,treatment_date,doctor_name,specialty,treatment_id_y,patient_feedback_score,review_date
0,T0001,P0067,D006,part,2723.53,1932.58,2025-08-04,John Morales,"Scientist, clinical (histocompatibility and im...",T035,4.5,2025-05-23
1,T0002,P0006,D022,machine,3963.16,2215.53,2024-09-29,Brandon Byrd,"Education officer, museum",T021,2.4,2025-04-19
2,T0003,P0637,D014,seat,4108.71,4958.21,2024-09-14,Antonio Stone,Nutritional therapist,T052,4.8,2025-06-06
3,T0004,P0314,D003,star,4793.77,1614.28,2024-09-25,Daniel Frye,Metallurgist,T089,1.8,2025-06-22
4,T0005,P0326,D012,art,4138.91,2779.05,2024-10-11,Cynthia Rodriguez,Automotive engineer,T100,4.2,2025-06-23


In [11]:

doctor_name_col = "doctor_name" if "doctor_name" in final_df.columns else None
group_cols = ["doctor_id"] + ([doctor_name_col] if doctor_name_col else [])

agg_dict = {}
if "patient_feedback_score" in final_df.columns:
    agg_dict["patient_feedback_score"] = ["mean", "count"]

agg_df = (
    final_df
    .groupby(group_cols, dropna=False)
    .agg(agg_dict)
)

agg_df.columns = ["avg_feedback_score", "feedback_count"]
agg_df = agg_df.reset_index()

print("Agg shape:", agg_df.shape)


Agg shape: (50, 4)


In [14]:
agg_df.head()

,doctor_id,doctor_name,avg_feedback_score,feedback_count
0,D001,Cheyenne Wilson,2.863636,22
1,D002,Taylor Potts,3.284211,19
2,D003,Daniel Frye,2.820000,15
3,D004,Crystal West,2.966667,15
4,D005,Jennifer Smith,2.371429,14


In [13]:
import os
os.makedirs("final_data", exist_ok=True)

In [15]:
processed_path = "final_data/processed_data.csv"
agg_df.to_csv(processed_path, index=False)